Import the needed libaries

In [1]:
import pandas as pd
import ast

Read in the CSV file and check the length

In [2]:
dataset = pd.read_csv("CSV Data/Dataset.csv")
print(f"Dataset Size: {len(dataset)}")
dataset.head()

Dataset Size: 7753


,ID,Unaligned Sequence,A,R,N,D,C,E,Q,G,...,Total Gaps in Alignment,Average Gap Length,Sequence Length_y,Gap Count,Percentage Gaps,Mutations from Consensus,Experimental,Resolution,R Value,R Free
0,1GNH_A,QTDMSRKAFVFPKESDTSYVSLKAPLTKPLKAFTVCLHFYTELSST...,0.043689,0.029126,0.033981,0.043689,0.009709,0.067961,0.033981,0.082524,...,33210307,3.164685,4497,4291,95.419168,4497,x-ray diffraction,3.0,0.239,0.273
1,1A3B_L,ADCGLRPLFEKKSLEDKTERELLESYI,0.037037,0.074074,0.000000,0.074074,0.037037,0.185185,0.000000,0.037037,...,33210307,3.164685,4497,4470,99.399600,4497,x-ray diffraction,1.8,0.170,0.230
2,1A3B_H,IVEGSDAEIGMSPWQVMLFRKSPQELLCGASLISDRWVLTAAHCLL...,0.043825,0.071713,0.039841,0.063745,0.027888,0.055777,0.031873,0.079681,...,33210307,3.164685,4497,4246,94.418501,4497,x-ray diffraction,1.8,0.170,0.230
3,1A3B_I,GDFEEIPEEYL,0.000000,0.000000,0.000000,0.090909,0.000000,0.363636,0.000000,0.090909,...,33210307,3.164685,4497,4486,99.755392,4497,x-ray diffraction,1.8,0.170,0.230
4,2A5E_A,MEPAAGSSMEPSADWLATAAARGRVEEVRALLEAGALPNAPNSYGR...,0.185897,0.108974,0.025641,0.070513,0.006410,0.070513,0.006410,0.089744,...,33210307,3.164685,4497,4341,96.531021,4497,solution nmr,NaN,NaN,NaN


Check which columns have missing values

In [3]:
print(f"Values Missing:\n{dataset.isna().any()}")

Values Missing:
ID                                       False
Unaligned Sequence                       False
A                                        False
R                                        False
N                                        False
D                                        False
C                                        False
E                                        False
Q                                        False
G                                        False
H                                        False
I                                        False
L                                        False
K                                        False
M                                        False
F                                        False
P                                        False
S                                        False
T                                        False
W                                        False
Y                                        Fal

Print the total number of values that are missing in the specified columns

In [4]:
print("Total Missing Values in Resolution: ", dataset["Resolution"].isna().sum())
print("Total Missing Values in R Value: ", dataset["R Value"].isna().sum())
print("Total Missing Values in R Free: ", dataset["R Free"].isna().sum())

Total Missing Values in Resolution:  878
Total Missing Values in R Value:  1601
Total Missing Values in R Free:  3205


Remove the rows where the missing values are in the specified columns

In [5]:
dataset.dropna(subset= ["Resolution", "R Value", "R Free"], inplace=True)
print(f"Dataset Size: {len(dataset)}")

Dataset Size: 4497


Check the column types and change them if needed

In [6]:
float_count, f_array = 0, []
int_count, i_array = 0, []
object_count, o_array = 0, []

for column in dataset.columns:
    if dataset[column].dtype == object:
        object_count += 1
        o_array.append(column)
    elif dataset[column].dtype == float:
        float_count += 1
        f_array.append(column)
    elif dataset[column].dtype == int:
        int_count += 1
        i_array.append(column)

print(f"Number of Columns: {len(dataset.columns)}\nNumber of Columns Checked: {object_count + int_count + float_count}\n")
print(f"Number of Object Columns: {object_count}")
print(f"Object Columns: {o_array}\n")
print(f"Number of Integer Columns: {int_count}")
print(f"Integer Columns: {i_array}\n")
print(f"Number of Float Columns: {float_count}")
print(f"Float Columns: {f_array}\n")

Number of Columns: 43
Number of Columns Checked: 43

Number of Object Columns: 7
Object Columns: ['ID', 'Unaligned Sequence', 'Hydrophobicity (Kyte-Doolittle Scale)', 'Aligned Sequence', 'Consensus Sequence', 'Percentage of Gaps Per Position', 'Experimental']

Number of Integer Columns: 5
Integer Columns: ['Sequence Length_x', 'Total Gaps in Alignment', 'Sequence Length_y', 'Gap Count', 'Mutations from Consensus']

Number of Float Columns: 31
Float Columns: ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', 'Net Charge at pH 7.0 (Neutral)', 'Net Charge at pH 3.0 (Acidic)', 'Net Charge at pH 11.0 (Basic)', 'Isoelectric Point', 'Molecular Weight', 'Conservation Scores', 'Average Gap Length', 'Percentage Gaps', 'Resolution', 'R Value', 'R Free']



Convert the columns of type object to the correct ones

In [7]:
def literal_eval(x):
    try:
        return ast.literal_eval(x)
    except ValueError:
        return None

for i in range(len(o_array)):
    if o_array[i] in ["ID", "Unaligned", "Aligned Sequence", "Consensus Sequence", "Experimental"]:
        dataset[o_array[i]] = dataset[o_array[i]].astype(str)
    else:
        dataset[o_array[i]] = dataset[o_array[i]].apply(literal_eval)

for j in range(len(o_array)):
    if o_array[i] not in ["ID", "Unaligned", "Aligned Sequence", "Consensus Sequence", "Experimental"]:
        dataset = dataset[dataset[o_array[i]].notnull()]

for column in dataset.columns:
    print(f"Column '{column}' has data type: {dataset[column].dtype}")

Column 'ID' has data type: object
Column 'Unaligned Sequence' has data type: object
Column 'A' has data type: float64
Column 'R' has data type: float64
Column 'N' has data type: float64
Column 'D' has data type: float64
Column 'C' has data type: float64
Column 'E' has data type: float64
Column 'Q' has data type: float64
Column 'G' has data type: float64
Column 'H' has data type: float64
Column 'I' has data type: float64
Column 'L' has data type: float64
Column 'K' has data type: float64
Column 'M' has data type: float64
Column 'F' has data type: float64
Column 'P' has data type: float64
Column 'S' has data type: float64
Column 'T' has data type: float64
Column 'W' has data type: float64
Column 'Y' has data type: float64
Column 'V' has data type: float64
Column 'Hydrophobicity (Kyte-Doolittle Scale)' has data type: object
Column 'Net Charge at pH 7.0 (Neutral)' has data type: float64
Column 'Net Charge at pH 3.0 (Acidic)' has data type: float64
Column 'Net Charge at pH 11.0 (Basic)' has

Get all the unique values in the experimental methods

In [8]:
print(dataset["Experimental"].unique())

['x-ray diffraction' 'electron crystallography' 'fiber diffraction']


Get the distribution of the resolution

In [9]:
dataset["Resolution"].describe()

count    4497.000000
mean        2.270469
std         0.452512
min         0.950000
25%         1.900000
50%         2.250000
75%         2.600000
max         4.750000
Name: Resolution, dtype: float64